In [14]:
from dotenv import load_dotenv
import fmpsdk
import pandas as pd
import os

load_dotenv()
apikey = os.environ.get("apikey")
snp500 = pd.DataFrame(fmpsdk.sp500_constituent(apikey))

PBR_KEY = 'priceBookValueRatio'

from fetch import Fetcher

file_directories = ['financial-ratios-quarter-20220820.csv']
key_list = [PBR_KEY]
key = key_list[0]

fetcher = Fetcher()
fetcher.register(key_list=key_list, csv_file_directory=file_directories[0])
from strategy import *

selector_small_pbr = Selector(type=SelectorType.HORIZON_RANGE, key=PBR_KEY, value=(0.0, 2.0), horizon=4)
selector_sequence = SelectorSequence(selectors=[selector_small_pbr])
filter_small_pbr = Filter(selector_sequence_union=[selector_sequence])

selector_large_pbr = Selector(type=SelectorType.HORIZON_RANGE, key=PBR_KEY, value=(3.0, 100.0), horizon=4)
selector_sequence = SelectorSequence(selectors=[selector_large_pbr])
filter_large_pbr = Filter(selector_sequence_union=[selector_sequence])


In [18]:
initial_holdings = [Asset(symbol=symbol) for symbol in snp500['symbol']]
start_date = '2017-03-01'
end_date = '2019-03-01'
rebalance_dates = ['2018-03-06', '2018-09-06']

filter_list = dict.fromkeys(['small-pbr', 'large-pbr'])
filter_list['small-pbr'] = filter_small_pbr
filter_list['large-pbr'] = filter_large_pbr

strategist_list = []
for key in filter_list.keys():
    filter_ = filter_list[key]
    strategist = Strategist(fetcher=fetcher, asset_pool=initial_holdings, start_date=start_date, end_date=end_date)
    for rebalance_date in rebalance_dates:
        strategist.select_assets(filter=filter_, rebalance_date=rebalance_date)
    strategist_list.append(strategist)

In [19]:
from analyze import Analyst

analyst = Analyst()
analyst.register(table_name='price_list', csv_file_directory='snp500_price_list_19620102_20220814.csv')
analyst.register(table_name='economic_indicators', csv_file_directory='snp500_price_list_19620102_20220814.csv')

analyst.attach('small-pbr', strategist_list[0])
analyst.attach('large-pbr', strategist_list[1])